# Reproduction of Tangri et al.'s Kidney Failure Risk Equation (KFRE)

## Import Libraries

In [1]:
########################### Import Requisite Libraries #########################
import pandas as pd
import numpy as np
import os
import sys
import pprint

################################################################################
# Add the parent directory to sys.path to access 'functions.py'
sys.path.append(os.path.join(os.pardir))
from python_scripts.functions import *
from python_scripts.kfre import *

## Read in the Data

In [2]:
base_path = os.path.join(os.pardir)

# Go up one level from 'notebooks' to the parent directory, then into the 'data' folder
data_path = os.path.join(os.pardir, "data")

image_path_png = os.path.join(base_path, "images", "png_images")
image_path_svg = os.path.join(base_path, "images", "svg_images")

# Use the function to ensure the 'data' directory exists
ensure_directory(data_path)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

# Read the CSV file into a DataFrame
df = pd.read_parquet(os.path.join(data_path, "df.parquet"))
df_eda = pd.read_parquet(os.path.join(data_path, "df_eda.parquet"))

Directory exists: ..\data
Directory exists: ..\images\png_images
Directory exists: ..\images\svg_images


In [3]:
# join back sex to original dataframe for eda dataframe so it can be used
df = df.join(df_eda["SEX"], on="Patient_ID", how="inner")

In [4]:
df_kfre = df.copy(deep=True)

In [5]:
df_kfre.columns

Index(['Age', 'Diabetes (1=yes; 0=no)', 'Hypertension (1=yes; 0=no)',
       'Renal disease (DM=1, HTN=2, GN=3, ADPKD=4, Other=5)', 'eGFR-EPI',
       'uPCR', 'Calcium (mmol/L)', 'Phosphate (mmol/L)',
       'Bicarbonate (mmol/L)', 'Albumin (g/l)', 'Follow-up YEARS', 'RIP',
       'ESRD', 'SEX_Female', 'SEX_Male',
       'ETHNICITY_AFRICAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_ANY OTHER ASIAN BACKGROUND',
       'ETHNICITY_ANY OTHER BLACK BACKGROUND',
       'ETHNICITY_ANY OTHER ETHNIC GROUP',
       'ETHNICITY_ANY OTHER MIXED BACKGROUND',
       'ETHNICITY_ANY OTHER WHITE BACKGROUND',
       'ETHNICITY_BANGLADESHI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_CARIBBEAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_CHINESE (OTHER ETHNIC GROUPS)',
       'ETHNICITY_INDIAN (ASIAN OR ASIAN BRITISH)', 'ETHNICITY_NOT STATED',
       'ETHNICITY_PAKISTANI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_WHITE AND BLACK AFRICAN (MIXED)', 'ETHNICITY_WHITE BRITISH',
       'ETHNICITY_WHITE IRIS

In [6]:
# subset the dataframe to include only relevant variables for the KFRE equation
# Age, eGFR, log-normalized uACR, Diabetes, Hypertension
df_kfre = df_kfre[
    [
        "Age",
        "SEX",
        "eGFR-EPI",
        "uACR",
        "Diabetes (1=yes; 0=no)",
        "Hypertension (1=yes; 0=no)",
        "Calcium (mmol/L)",
        "Phosphate (mmol/L)",
        "Bicarbonate (mmol/L)",
        "ESRD",
    ]
]

In [7]:
df_kfre["Region"] = "Non North American"

In [8]:
columns = {
    "age": "Age",
    "sex": "SEX",
    "eGFR": "eGFR-EPI",
    "uACR": "uACR",
    "region": "Region",
    "dm": "Diabetes (1=yes; 0=no)",
    "htn": "Hypertension (1=yes; 0=no)",
}

# Assuming `df` is your DataFrame loaded with the dataset
predictor = RiskPredictor(data=df_kfre, columns=columns)
risk = predictor.predict(2, use_extra_vars=True)


# Predict risk for different combinations of years and variables
pred_4var_2year = predictor.predict(years=2)
pred_4var_5year = predictor.predict(years=5)
pred_6var_2year = predictor.predict(years=2, use_extra_vars=True)
pred_6var_5year = predictor.predict(years=5, use_extra_vars=True)

# inner join the different combinations of years and variables to
# kfre_inclusion dataframe
df_kfre = df_kfre.assign(
    pred_4var_2year=pred_4var_2year,
    pred_4var_5year=pred_4var_5year,
    pred_6var_2year=pred_6var_2year,
    pred_6var_5year=pred_6var_5year,
)

In [9]:
df_kfre

,Age,SEX,eGFR-EPI,uACR,Diabetes (1=yes; 0=no),Hypertension (1=yes; 0=no),Calcium (mmol/L),Phosphate (mmol/L),Bicarbonate (mmol/L),ESRD,Region,pred_4var_2year,pred_4var_5year,pred_6var_2year,pred_6var_5year
Patient_ID,,,,,,,,,,,,,,,
568268916,87.24,Male,19.0,5.744563,1.0,1.0,2.78,0.88,27.2,0,Non North American,0.018785,0.070800,0.017863,0.067413
659549633,56.88,Female,15.0,140.661958,0.0,1.0,2.43,1.02,21.3,1,Non North American,0.173785,0.522508,0.189643,0.557029
406748956,66.53,Female,17.0,35.224504,0.0,1.0,2.33,1.24,27.8,0,Non North American,0.064027,0.226029,0.070294,0.245901
109228713,69.92,Male,12.0,74.299919,0.0,1.0,2.29,1.80,20.7,1,Non North American,0.174712,0.524577,0.190644,0.559143
220533110,81.14,Female,15.0,59.683881,1.0,1.0,2.45,1.39,26.2,0,Non North American,0.073213,0.255029,0.069716,0.244084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708268897,68.45,Female,23.0,310.140610,0.0,1.0,2.38,0.53,27.8,0,Non North American,0.083089,0.285301,0.091128,0.309260
019128787,32.26,Female,16.0,284.328640,0.0,1.0,2.46,1.51,21.3,1,Non North American,0.331894,0.790219,0.358696,0.820976
371958631,72.34,Male,20.0,307.612850,1.0,1.0,2.51,0.96,22.9,0,Non North American,0.132200,0.422506,0.126083,0.406582
